In [23]:
# Cabeçalhos para uso do Pyomo
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [24]:
#conjuntos
cidades = [0,1,2,3,4,5,6]
caminhao = [1,2,3]
sub1 = [0,6]
sub2 = [1,2]
sub3 = [3,4,5]
sub4 = [3,4]
#parâmetros 
custo = {
     (0,0):0,(0,1):12,(0,2):9,(0,3):17,(0,4):13,(0,5):10,
     (1,0):14,(1,1):0,(1,2):5,(1,3):17,(1,4):17,(1,5):19,
     (2,0):10,(2,1):4,(2,2):0,(2,3):13,(2,4):11,(2,5):15,
     (3,0):16,(3,1):16,(3,2):12,(3,3):0,(3,4):7,(3,5):10,
     (4,0):9,(4,1):18,(4,2):14,(4,3):11,(4,4):0,(4,5):6,
     (5,0):11,(5,1):20,(5,2):16,(5,3):14,(5,4):9,(5,5):0
}
demanda = {
    0:0,1:310,2:220,3:180,4:290,5:370,6:0
}
capacidade = {1:1000, 2:1000,3:1000}


In [28]:
#PYOMO
model = pyo.ConcreteModel()

model.I = pyo.Set(initialize = cidades)
model.C =pyo.Set(initialize = caminhao)
model.sub1 = pyo.Set(initialize = sub1)
model.sub2 = pyo.Set(initialize = sub2)
model.sub3 = pyo.Set(initialize = sub3)
model.sub4 = pyo.Set(initialize = sub4)

model.c = pyo.Param(model.I*model.I, initialize = custo, within = pyo.NonNegativeReals)
model.d = pyo.Param(model.I, initialize = demanda, within = pyo.NonNegativeReals)
model.cap = pyo.Param(model.C, initialize = capacidade, within=pyo.NonNegativeReals)


model.x = pyo.Var(model.I*model.I*model.C, within = pyo.Binary)
model.u = pyo.Var(model.I*model.C, within = pyo.NonNegativeReals)

model.constraints = pyo.ConstraintList()
#model.constraints.add(expr = model.u[1] == 0)
for i in model.I:
  if i > 0 and i <  len(cidades)-1:
    model.constraints.add(expr = sum(sum(model.x[i,j,k] for j in model.I if i!=j if j>0) for k in model.C ) == 1)
for j in model.I:
  if j > 0 and j <  len(cidades)-1:
    model.constraints.add(expr = sum(sum(model.x[i,j,k] for i in model.I if i!=j if i<len(cidades) -1) for k in model.C ) == 1)
#for i in model.I:
  #for j in model.I:
    #if i != j and j != 1:
      #model.constraints.add(expr = model.u[j] >= model.u[i]+ model.x[i,j,k] - 100*(1-model.x[i,j,k]))
for k in model.C:
  for j in model.I:
    if j > 0 and j <  len(cidades)-1:
      model.constraints.add(expr = sum(model.x[i,j,k]for i in model.I if i != j and i < len(cidades)-1) == sum(model.x[j,h,k]for h in model.I if h != j and h > 0))
for k in model.C:
   model.constraints.add(expr = sum(sum(model.x[i,j,k]*model.d[i] for j in model.I if j != i if j > 0) for i in model.I if i <  len(cidades)-1) <= model.cap[k])
   model.constraints.add(expr = sum(model.x[0,j,k] for j in model.I if j > 0) == 1)
   model.constraints.add(expr = sum(model.x[i, len(cidades)-1,k] for i in model.I if i <  len(cidades)-1) == 1)
   model.constraints.add(expr = sum(sum(model.x[i,j,k] for i in model.sub1)for j in model.sub1 if j != i) <= len(sub1) -1)
   model.constraints.add(expr = sum(sum(model.x[i,j,k] for i in model.sub2)for j in model.sub2 if j != i) <= len(sub2) -1)
   model.constraints.add(expr = sum(sum(model.x[i,j,k] for i in model.sub3)for j in model.sub3 if j != i) <= len(sub3) -1)
   model.constraints.add(expr = sum(sum(model.x[i,j,k] for i in model.sub4)for j in model.sub4 if j != i) <= len(sub4) -1)


model.func = pyo.Objective(expr = sum(sum(sum(model.x[i,j,k]*model.c[i,j] for j in model.I if j != i if j < len(cidades) -1) for i in model.I if i < len(cidades) -1) for k in model.C))


solver = SolverFactory('glpk')
resultado = solver.solve(model, tee=False)

if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
  print(f'distancia minima: {pyo.value(model.func)}')
  for i in model.I:
    if i < len(cidades) -1:
      for j in model.I:
        if j > 0:
          for k in model.C:
            #if custo.get((i,j)) != 0 :
              if pyo.value(model.x[i,j,k]) == 1:
                print(f' o caminhão {k} vai de {i} para {j}')

distancia minima: 43.0
 o caminhão 3 vai de 0 para 2
 o caminhão 2 vai de 0 para 3
 o caminhão 1 vai de 0 para 6
 o caminhão 3 vai de 1 para 6
 o caminhão 3 vai de 2 para 1
 o caminhão 2 vai de 3 para 4
 o caminhão 2 vai de 4 para 5
 o caminhão 2 vai de 5 para 6


In [29]:
model.constraints.pprint()

constraints : Size=46, Index=constraints_index, Active=True
    Key : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                 : Upper  : Active
      1 :   1.0 :                                                                                                                                                                                                                   x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,2,2] + x[1,3,2] + x[1,4,2] + x[1,5,2] + x[1,6,2] + x[1,2,3] + x[1,3,3] + x[1,4,3] + x[1,5,3] + x[1,6,3] :    1.0 :   True
      2 :   1.0 :                                                                                                         